In [60]:
from pymongo import UpdateOne
from tqdm import tqdm
from exploration.config import mongo_inst
from mlpp.data_collection.pp_calculation import fast_pp_hist

In [3]:
osu_random_db = mongo_inst['osu_random_db']

In [72]:
user_ids = list(map(lambda c: c['_id'], osu_random_db['osu_user_stats'].find({}, {})))

In [73]:
for user_id in tqdm(user_ids):
    scores = list(osu_random_db['osu_scores_high'].find({'user_id': user_id}, {'pp': 1, 'date': 1}).sort('date'))
    score_hist = list(map(lambda x: x['pp'] if x['pp'] else 0, scores))
    pp_hist = fast_pp_hist(score_hist)

    score_updates = []
    for i in range(len(scores)):
        query = {'_id': scores[i]['_id']}
        update = {'$set': {'mlpp.est_user_pp': pp_hist[i]}}
        score_updates.append(UpdateOne(query, update))

    osu_random_db['osu_scores_high'].bulk_write(score_updates)
 

100%|██████████| 59697/59697 [28:51<00:00, 34.48it/s] 
